In [ ]:
import numpy as np
import scipy
from scipy.stats import qmc

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from LCDM.LCDM import CobraLCDM
from GEN.GEN import CobraGEN
from Utils.no_wiggle_power import no_wiggle_power
from Utils.sigma_squared_bao import sigma_squared_bao
from Utils.ir_resummed_power import ir_resummed_power
from Utils.convert_units import convert_units_k_pk_from_hfid, convert_units_k_pk_to_hfid
from scipy.interpolate import CubicSpline

### Linear Matter Power Spectrum: Comparison to CAMB

In the first part of this demonstration notebook, we'll be running COBRA predictions for the linear matter power spectrum and comparing to CAMB. (Our decomposition is based on CAMB 1.5.)

In [ ]:
import camb 
from camb import model, initialpower 
from camb.dark_energy import DarkEnergyPPF, DarkEnergyFluid
from camb.reionization import TanhReionization 

The linear power spectrum can be computed either in LCDM or for more generalized model. Let's look at the former first; there are six parameters in this case.

In [ ]:
## instantiate object of class

cobra_lcdm = CobraLCDM(param_range = 'def')

## specify wavenumbers and cosmology (these are interpreted in units of h_fid / Mpc where h_fid = 0.7, hence the notation)
k_min_hfid = 0.002
k_max_hfid = 1.5
n_bins = 300
k_out_hfid = np.logspace(np.log10(k_min_hfid), np.log10(k_max_hfid), n_bins)
params = {'omch2':[0.13], 'ombh2':[0.021], 'ns':[0.97], 'As': [2], 'h': [0.68], 'z':[1]} # you pass a dict (more on vectorization below)

## by default, a call requires three arguments: the cosmological parameters, the output bins and the number of basis functions

## compute linear power spectrum with a different number of basis functions
plin_cobra_nb12 = cobra_lcdm.linear_matter_power(params, k_out_hfid, n_basis_list = [12])[0] ## you get back an array of size #cosmo x #bins
plin_cobra_nb5 = cobra_lcdm.linear_matter_power(params, k_out_hfid, n_basis_list = [5])[0]

cosmo = np.array([val for val in params.values()]).T

## let's compute the linear power spectrum using CAMB as well:
omega_c, omega_b, n_s, A_s, h, z = cosmo[0]
A_s = A_s*10**(-9)
pars = camb.set_params(kmax = 4) 
pars.set_cosmology(H0=100*h, ombh2=omega_b, omch2=omega_c,mnu=0,TCMB=2.7255)
pars.InitPower.set_params(ns=n_s,As=A_s)
pars.set_accuracy(AccuracyBoost=3)
pars.set_matter_power(redshifts=[z], kmax=4.0,accurate_massive_neutrino_transfers=True)
results = camb.get_results(pars)

## note kh is in units of h/Mpc, not hfid/Mpc. 
kh, z, pk1 = results.get_matter_power_spectrum(minkh=k_min_hfid*0.7/h, maxkh=k_max_hfid*0.7/h, npoints = n_bins)
p_out_camb = pk1[0]*(0.7/h)**3


In [ ]:
plt.xscale('log')
plt.plot(k_out_hfid, p_out_camb/plin_cobra_nb12)
plt.plot(k_out_hfid, p_out_camb/plin_cobra_nb5)
plt.xlabel('k (h*/Mpc)',fontsize=16)
plt.ylabel('CAMB / COBRA',fontsize=16)
plt.legend(['12 basis functions','5 basis functions'],loc=[0.1,0.8])
plt.show()

Evaluation is fast, even with maximum (default) number of basis functions:

In [ ]:
%%timeit
plin_cobra = cobra_lcdm.linear_matter_power(params, k_out_hfid, n_basis_list=[12])[0]

In more generalized cases, you can pass 10 cosmological parameters. They include massive neutrinos, dynamical dark energy and curvature. 

In [ ]:
cobra_gen = CobraGEN(param_range = 'ext')

params_gen = {'omch2':[0.12], 'ombh2':[0.023],'Omk':[0.15],'h':[0.75],'ns':[0.99],'Mnu':[0.5],'w0':[-0.9],'wp':[-0.6],'z':[1],'As':[4]} 
plin_cobra_gen_nb14 = cobra_gen.linear_matter_power(params_gen, k_out_hfid, n_basis_list=[14])[0] ## use same k range as before
plin_cobra_gen_nb6 = cobra_gen.linear_matter_power(params_gen, k_out_hfid, n_basis_list=[6])[0]


In [ ]:
cosmo_gen = np.array([val for val in params_gen.values()]).T
omega_c,omega_b,omega_k,h,n_s,m_nu,w0,wp,z,A_s = cosmo_gen[0]
pars = camb.set_params(num_nu_massless = 2.044,kmax = 4.3464234,DoLensing=False)

pars.set_cosmology(H0=100*h, ombh2=omega_b, omch2=omega_c,mnu=m_nu,omk=omega_k,TCMB=2.7255)
pars.InitPower.set_params(ns=n_s,As=A_s*10**(-9))
pars.set_accuracy(AccuracyBoost=2)
pars.DarkEnergy=DarkEnergyPPF(w=w0, wa=wp-w0) ##plug in the correct value of wa = wp - w0
pars.set_matter_power(redshifts=[z], kmax=4.0,accurate_massive_neutrino_transfers=True)
results = camb.get_results(pars)
results.Params.Reion = TanhReionization(Reionization=True)
results = camb.get_results(pars)
kh, z, pk1 = results.get_matter_power_spectrum(k_min_hfid*0.7/h, maxkh=k_max_hfid*0.7/h, npoints = n_bins)
## note kh is in units of h/Mpc, not hfid/Mpc. 
p_out_camb_gen = pk1[0]*(0.7/h)**3

In [ ]:
plt.xscale('log')
plt.plot(k_out_hfid, p_out_camb_gen/plin_cobra_gen_nb14)
plt.plot(k_out_hfid, p_out_camb_gen/plin_cobra_gen_nb6)
plt.xlabel('k (h*/Mpc)',fontsize=16)
plt.ylabel('CAMB / COBRA',fontsize=16)
plt.legend(['14 basis functions','6 basis functions'],loc=[0.1,0.8])
plt.show()

In [ ]:
%%timeit
plin_cobra_gen = cobra_gen.linear_matter_power(params_gen, k_out_hfid, n_basis_list = [14], resum = True, disps_hfid = np.array([30]))[0]


In [ ]:
## you can also pass hundreds of cosmologies at once: let's do this for the default range of the extended parameter space
## syntax is similar to BACCO here: see e.g. https://bacco.dipc.org/

l_bounds_tot = np.array([0.095,0.0202,-0.12,0.55,0.9,0,-1.25,-0.3,0.1,1]) ## parameter bounds for the default range
u_bounds_tot = np.array([0.145,0.0238,0.12,0.8,1.02,0.6,-0.75,0.3,3,5])

cobra_gen_def = CobraGEN(param_range = 'def')

In [ ]:
%%timeit
ncos = 250
params_unit_cube = np.random.uniform(0,1,(ncos,10))
params_gen_array = qmc.scale(params_unit_cube,l_bounds_tot,u_bounds_tot) 
params_gen = {'omch2':params_gen_array[:,0],
              'ombh2':params_gen_array[:,1],
              'Omk':params_gen_array[:,2],
              'h':params_gen_array[:,3],
              'ns':params_gen_array[:,4],
              'Mnu':params_gen_array[:,5],
              'w0':params_gen_array[:,6],
              'wa':params_gen_array[:,7],
              'z':params_gen_array[:,8],
              'As':params_gen_array[:,9]} 
              
    
plin_cobra_gen = cobra_gen_def.linear_matter_power(params_gen, k_out_hfid, n_basis_list = [12])[0]

COBRA has some additional options for the linear power spectrum. For example, you can also request resummation via the wiggle-no-wiggle split. In LCDM, the displacement is emulated automatically (consistently with the wiggle-no-wiggle split).

In [ ]:
plin_cobra_noresum = cobra_lcdm.linear_matter_power(params, k_out_hfid, n_basis_list = [12])[0] 
plin_cobra_resum = cobra_lcdm.linear_matter_power(params, k_out_hfid, n_basis_list = [12], resum = True)[0] 

In [ ]:
plt.xscale('log')
plt.plot(k_out_hfid, plin_cobra_resum/plin_cobra_noresum)
plt.xlabel('k (h*/Mpc)',fontsize=16)
plt.ylabel(r'$P_{resum} / P$',fontsize=16)
plt.show()

In extended cosmologies, the displacement is not emulated (yet) - you have to provide it yourself in the call. This is under construction. You can, however, compute the displacement beforehand yourself using the built-in brute-force integration via compute_disp. You can also directly apply compute_pir to a given input linear spectrum (though this is not vectorized)

In [ ]:
k_pk = np.array([k_out_hfid, plin_cobra_gen_nb14])
plin_gen_nw = no_wiggle_power(k_pk)[1]

In [ ]:
plt.xscale('log')
plt.plot(k_out_hfid, plin_cobra_gen_nb14/plin_gen_nw)
plt.xlabel('k (h*/Mpc)',fontsize=16)
plt.ylabel(r'$P / P_{nw}$',fontsize=16)
plt.show()
## it is still advisable to inspect the result of the no-wiggle calculation before using it for the more general cosmologies 

In [ ]:
## you can also compute the displacement if you want to: 
displ = sigma_squared_bao(k_pk)
displ

### Galaxy Power Spectrum in Redshift Space

In LCDM, we can also compute the power spectrum of galaxies as implemented in the EPT module of velocileptors. Let's see how that works! 
For the theory model and implementation of velocileptors we use here, see https://github.com/sfschen/velocileptors/blob/master/velocileptors/EPT/ept_fullresum_fftw.py
    and https://arxiv.org/pdf/2005.00523,  https://arxiv.org/abs/2012.04636.

In [ ]:
## some of these values taken from velocileptors demo notebook

bias_params_ept = {'b1':[1.68593608], 'b2':[-1.17], 'bs':[-0.715], 'b3':[-0.479], \
               'alpha0':[50], 'alpha2':[50], 'alpha4':[50], 'alpha6':[50], \
               'sn':[3000], 'sn2':[3000], 'sn4':[3000], 'bfog':[0]}

ept_cosmo = np.loadtxt("test_velocileptors_cosmo.dat") ## a test cosmology; note that A_s is fixed to 2e-9 as in the paper
ept_pell = np.loadtxt("test_velocileptors.dat") ## test example from velocileptors - monopole, quadrupole and hexadecapole
params_ept = {'omch2':[ept_cosmo[0]], 'ombh2':[ept_cosmo[1]], 'ns':[ept_cosmo[2]], 
              'As': [ept_cosmo[3]], 'h': [ept_cosmo[4]], 'z':[ept_cosmo[5]]}

k_out_loop = cobra_lcdm.s_tables_loop["k_loop"] ## the test example from velocileptors is computed at k_loop wavenumbers

params_ept

In [ ]:
%%timeit 
pell = cobra_lcdm.oneloop_galaxy_power_multipoles(params_ept, bias_params_ept, k_out_loop, n_basis_list = [12,12]) 

In [ ]:
## the default number of basis functions for this parameter range is 12
cobra_pell_nb12 = cobra_lcdm.oneloop_galaxy_power_multipoles(params_ept, bias_params_ept, k_out_loop, n_basis_list = [12,12])[0]

## for comparison, use 6 for linear and 6 for loop calculation
cobra_pell_nb6 = cobra_lcdm.oneloop_galaxy_power_multipoles(params_ept, bias_params_ept, k_out_loop, n_basis_list = [6,6])[0] 

In [ ]:
## let's look at the monopole: 
plt.xscale('log')
plt.plot(k_out_loop, cobra_pell_nb12[0]/ept_pell[0])
plt.plot(k_out_loop, cobra_pell_nb6[0]/ept_pell[0])
plt.xlabel('k (h*/Mpc)',fontsize=16)
plt.ylabel(r'$P^0_{cobra} / P^0_{velocileptors}$',fontsize=16)
plt.legend(['12 basis functions','6 basis functions'],loc=[0.1,0.8])
plt.show()

In [ ]:
## you can also convert power spectra back to h/Mpc units:

k_p0 = np.array([k_out_loop, cobra_pell_nb12[0]])
k_p0_h = convert_units_k_pk_from_hfid(k_p0, h_out = ept_cosmo[4])

### Using your own weights

If you don't want to make any use of the built-in RBF emulators (which may be the case if you want to exceed their ranges, for example) then there is also the option of providing your own set of weights for the linear or one-loop calculation. Let's do an example!

In [ ]:
## call CAMB for a cosmology with a nonzero neutrino mass and a nontrivial DE equation of state
h=0.65
pars = camb.set_params(kmax = 20) 
pars.set_cosmology(H0=100*h, ombh2=0.022, omch2=0.12, mnu=0.6,TCMB=2.7255)
pars.InitPower.set_params(ns=n_s,As=2.2e-9)
pars.set_accuracy(AccuracyBoost=3)
pars.DarkEnergy=DarkEnergyPPF(w=-0.5, wa=0)
pars.set_matter_power(redshifts=[1], kmax=20, accurate_massive_neutrino_transfers=True)
results = camb.get_results(pars)

kh, z, pk1 = results.get_matter_power_spectrum(minkh=1e-4, maxkh=20, npoints = 600)
p_out_camb_mnuw0 = pk1[0]

In [ ]:
## we can try to decompose this power spectrum into the cobra basis
k_pk_mnu = np.array([kh,p_out_camb_mnuw0])
cobra_lcdm_ext = CobraLCDM(param_range = 'ext')
wts, proj = cobra_lcdm_ext.project_pk_into_basis(k_pk_mnu, h, n_basis_list=[12])


In [ ]:
## the projection is again in units of h_fid / Mpc, so to compare to the input we have to convert units again
proj_h = convert_units_k_pk_from_hfid(proj, h_out = h) ## gives back k_pk array

## let's compare them on the range of proj (which is narrower):
camb_spl = CubicSpline(np.log10(kh),p_out_camb_mnuw0)(np.log10(proj_h[0]))

In [ ]:
## using the maximum number of 16 available basis functions, we can approximate the power spectrum with neutrinos quite well

plt.xscale('log')
plt.plot(proj_h[0],camb_spl / proj_h[1])
plt.xlabel('k (h*/Mpc)',fontsize=16)
plt.ylabel('CAMB / COBRA',fontsize=16)
plt.show()


In [ ]:
## you can now use the weights as inputs for a loop calculation, putting the cosmology argument to 'None'.
## this of course still assumes that you can model all the dependence on cosmological parameters only via the input power spectrum
## let's put the growth rate equal to unity for simplicity

pell_mnuw0 = cobra_lcdm_ext.oneloop_galaxy_power_multipoles(cosmo = None, 
                                                          weights = np.array([wts]), 
                                                          bias = bias_params_ept,
                                                            n_basis_list = [12,12],
                                                          k_out_hfid = k_out_loop, 
                                                          growth_rates = np.array([1]),
                                                         resum = False)[0]

plt.xscale('log')
plt.plot(k_out_loop, pell_mnuw0[0])
plt.plot(k_out_loop, pell_mnuw0[1])
plt.plot(k_out_loop, 10*pell_mnuw0[2])
plt.legend([r'$P_0$',r'$P_2$',r'$10 P_4$'],loc=[0.6,0.6],fontsize = 16)
plt.xlabel('k (h*/Mpc)',fontsize=16)
plt.ylabel('P(k)',fontsize=16)
plt.show()

### Running a Chain: Minimal Working Example 

If you want to run an MCMC chain using COBRA, it's as simple as putting the 'compute' call in your likelihood. The only important thing to remember is that you should create the COBRA instance outside of the likelihood - creating the instance is slow, since all the requisite tables need to be loaded into memory. Here is an example of COBRA to perform cosmological inference on the one-loop matter power spectrum in LCDM.

In [ ]:
import dynesty
import getdist
from getdist import plots, MCSamples
import time

First we need to create some mock data. In practice this data_vec will contain real data.

In [ ]:
csq = 10 # speed of sound 
ctr = {'csq':[10]}
## let's assume we have a true data vector which is given to us in units of h_true / Mpc

boxlen = 2000 # box side length in units of Mpc/h_true 
kf = 2 * np.pi / boxlen
dk = 0.005 ## some arbitrary choice of binning
h_true = 0.68 #use params from above 
k_bins_h_loop = np.arange(0.005, 0.25, dk) # k range in units of h_true / Mpc
# Convert to the units used in this program
k_bins_hfid_loop = k_bins_h_loop * h_true / 0.7 

n_mode = 4 * np.pi * dk * k_bins_h_loop ** 2 / kf ** 3
frac_error = np.sqrt(2 / n_mode) ## fractional error on power spectrum in the Gaussian approximation

cobra_lcdm_ext = CobraLCDM(param_range = 'ext') 
## true vector in h_true units; let's use the cosmology from above 
data_vec = cobra_lcdm_ext.oneloop_matter_power(params, ctr, k_bins_hfid_loop, 
                                               n_basis_list = [12,12], resum = False)[0] * (h_true / 0.7) ** 3 

Now we will create the functions that will be used to fit the data.

In [ ]:
bounds = np.array([[0.08, 0, 0.5, -100], [0.175, 5, 0.9, 100]]) # w_cdm, A_s, h + last entry is the counterterm

def ptform(u):
    return bounds[0] + (bounds[1] - bounds[0]) * u

def loglike(cosmo_and_ctr):
    omch2, As, h, csq = cosmo_and_ctr
    params = {'omch2':[omch2], 'ombh2':[0.021], 'ns':[0.97], 'As':[As], 'h':[h],'z':[1]} # fix baryon density, spectral index and redshift
    k_bins_hfid_loop = k_bins_h_loop * h / 0.7
    ctr = {'csq':[csq]}
    pred = cobra_lcdm_ext.oneloop_matter_power(params, ctr, k_bins_hfid_loop, n_basis_list = [12,12], resum = False)[0] * (h / 0.7) ** 3
    return -0.5 * np.sum(((pred - data_vec) / (data_vec * frac_error)) ** 2)


run the sampler: this should take < 1min or so on a personal laptop

In [ ]:
start = time.time()
ndim = 4
sampler = dynesty.DynamicNestedSampler(loglike, ptform, ndim, nlive=500, sample='unif')
sampler.run_nested(dlogz_init=0.0001, maxcall=10**6, print_progress=False) 
res1loop=sampler.results
end = time.time()
print(end-start)

In [ ]:
samps1loop = res1loop.samples
names = ["x%s"%i for i in range(ndim)]
labels =  [r'\omega_{cdm}',r'A_s','h']
getdist_mcsamps = MCSamples(samples = samps1loop, names = names, labels = labels, weights = np.exp(res1loop.logwt-np.max(res1loop.logwt)))

g = plots.get_subplot_plotter()
g.triangle_plot([getdist_mcsamps], ['x0', 'x1', 'x2'], 
                filled=True, markers = {'x0':0.13,'x1':2,'x2':0.68})     